In [17]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html


### Step 1: Install and Import H2O

In [23]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,25 days 22 hours 25 mins
H2O_cluster_timezone:,America/Chicago
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,4 months and 20 days
H2O_cluster_name:,H2O_from_python_christinadong_8atvgy
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.917 Gb
H2O_cluster_total_cores:,14
H2O_cluster_allowed_cores:,14
H2O_cluster_status:,"locked, healthy"


### Step 2: Load Train/Test Data into H2O

In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [25]:
df = pd.read_csv('/Users/christinadong/Documents/MLops/mlops-final/data/healthcare-dataset-stroke-data.csv')
#  Train-Test Split
# ============================
# Drop ID column if it exists
if 'id' in df.columns:
    df = df.drop(columns=['id'])
    
target_col = 'stroke'
X = df.drop(columns=[target_col])
y = df[target_col]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("\nTrain set shape:", X_train.shape)
print("Test set shape:", X_test.shape)


Train set shape: (4088, 10)
Test set shape: (1022, 10)


In [26]:
# Join X and y so H2OFrame has the target in the same table
train_pd = pd.concat([X_train.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)
test_pd  = pd.concat([X_test.reset_index(drop=True),  y_test.reset_index(drop=True)],  axis=1)

train_h2o = h2o.H2OFrame(train_pd)
test_h2o  = h2o.H2OFrame(test_pd)

# Cast columns: make target categorical; make string/object columns categorical too
y = target_col
x = [c for c in train_h2o.columns if c != y]

# If this is classification, ensure factor
train_h2o[y] = train_h2o[y].asfactor()
test_h2o[y]  = test_h2o[y].asfactor()

# Optional: make object/string columns categorical
for c in x:
    if train_pd[c].dtype == "object":
        train_h2o[c] = train_h2o[c].asfactor()
        test_h2o[c]  = test_h2o[c].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


### Step 3: Run AutoML

In [27]:
aml = H2OAutoML(
    max_runtime_secs=600, 
    seed=42,
    nfolds=5,
    balance_classes=True,
    sort_metric="AUCPR",
    stopping_metric="AUCPR",
    verbosity="info"
)
aml.train(x=x, y=y, training_frame=train_h2o)

AutoML progress: |
19:49:41.290: Project: AutoML_6_20250816_194941
19:49:41.290: Setting stopping tolerance adaptively based on the training frame: 0.015640281177246362
19:49:41.290: Build control seed: 42
19:49:41.290: training frame: Frame key: AutoML_6_20250816_194941_training_py_27_sid_b7e9    cols: 11    rows: 4088  chunks: 1    size: 44184  checksum: 4140185812486358258
19:49:41.290: validation frame: NULL
19:49:41.290: leaderboard frame: NULL
19:49:41.290: blending frame: NULL
19:49:41.290: response column: stroke
19:49:41.290: fold column: null
19:49:41.290: weights column: null
19:49:41.291: AutoML: XGBoost is not available; skipping it.
19:49:41.291: Loading execution steps: [{XGBoost : [def_2 (1g, 10w), def_1 (2g, 10w), def_3 (3g, 10w), grid_1 (4g, 90w), lr_search (6g, 30w)]}, {GLM : [def_1 (1g, 10w)]}, {DRF : [def_1 (2g, 10w), XRT (3g, 10w)]}, {GBM : [def_5 (1g, 10w), def_2 (2g, 10w), def_3 (2g, 10w), def_4 (2g, 10w), def_1 (3g, 10w), grid_1 (4g, 60w), lr_annealing (6g, 10w

Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_grid_1_AutoML_6_20250816_194941_model_188


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    42                 42                          8352                   4            4            4             7             15            11.2143

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.36360634847354484
RMSE: 0.6029978013836741
LogLoss: 1.0481216055088176
Mean Per-Class Error: 0.16457575418480108
AUC: 0.9096188190919413
AUCPR: 0.8885603582563387
Gini: 0.8192376381838826

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.04511489093258007
       0     1     Error    Rate
-----  ----  ----  -------  ---------------
0      2882  1007  0.2589   (1007.0/3889.0)
1      273   3615  0.0702   (273.0/3888.0)
Total  3155  4622  0.1646   (1280.0/7777.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.0451149    0.849589  290
max f2                       0.0188047    0.916955  347
max f0point5                 0.106022     0.836413  188
max accuracy                 0.0652451    0.836569  254
max precision                0.453399     1         0
max recall                   0.0121394    1         361
max specificity              0.453399     1         0
max absolute_mcc             0.0451149    0.683112  290
max min_per_class_accuracy   0.0697953    0.825919  246
max mean_per_class_accuracy  0.0652451    0.836572  254
max tns                      0.453399     3889      0
max fns                      0.453399     3869      0
max fps                      0.00110864   3889      399
max tps                      0.0121394    3888      361
max tnr                      0.453399     1         0
max fnr                      0.453399     0.995113  0
max fpr                      0.00110864   1         399
max tpr                      0.0121394    1         361

Gains/Lift Table: Avg response rate: 49.99 %, avg score:  9.43 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score       cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100296                   0.373857           1.97461   1.97461            0.987179         0.40152     0.987179                    0.40152             0.0198045       0.0198045                  97.4613   97.4613            0.0195474
2        0.0203163                   0.342976           1.92525   1.94962            0.9625           0.355884    0.974684                    0.378413            0.0198045       0.0396091                  92.5248   94.9618            0.0385805
3        0.0304745                   0.327889           2.00026   1.9665             1                0.335648    0.983122                    0.364158            0.0203189       0.059928                   100.026   96.6498            0.0588994
4        0.0405041                   0.317127           2.00026   1.97486            1                0.319355    0.987302                    0.353064            0.0200617       0.0799897                  100.026   97.4857            0.0789612
5        0.0514337                   0.281831           1.85906   1.95025            0.929412         0.301383    0.975 

### Step 4: Leaderboard + pick the best model

In [28]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)
leader = aml.leader
leader.model_id

'GBM_grid_1_AutoML_6_20250816_194941_model_188'

### Step 5: Evaluate on test set

In [29]:
perf = leader.model_performance(test_h2o)
print(perf)                    

# Pull common metrics
test_auc    = perf.auc()
test_logloss= perf.logloss()
test_f1_opt = perf.F1()  # best F1 across thresholds
# Find the row with maximum F1
best_f1_row = max(test_f1_opt, key=lambda x: x[1])  # x[1] = F1 value

thresh = best_f1_row[0]
test_f1_opt = best_f1_row[1]
print({"AUC": test_auc, "LogLoss": test_logloss, "Best F1": test_f1_opt, "Threshold": thresh})

# Confusion matrix at F1-optimal threshold
cm = perf.confusion_matrix(metrics="f1", thresholds=[thresh])
cm

ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.04162539499875424
RMSE: 0.2040230256582679
LogLoss: 0.16167443264747958
Mean Per-Class Error: 0.22635802469135802
AUC: 0.8373559670781893
AUCPR: 0.2677956860964926
Gini: 0.6747119341563785

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.08378066947222473
       0    1    Error    Rate
-----  ---  ---  -------  --------------
0      843  129  0.1327   (129.0/972.0)
1      16   34   0.32     (16.0/50.0)
Total  859  163  0.1419   (145.0/1022.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.0837807    0.319249  140
max f2                       0.0814816    0.469169  147
max f0point5                 0.236792     0.357143  11
max accuracy                 0.293372     0.95499   5
max precision                0.386469     1         0
max recall                   

[Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.08378066947222473
       0    1    Error    Rate
-----  ---  ---  -------  --------------
0      843  129  0.1327   (129.0/972.0)
1      16   34   0.32     (16.0/50.0)
Total  859  163  0.1419   (145.0/1022.0),
 Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.08378066947222473
       0    1    Error    Rate
-----  ---  ---  -------  --------------
0      843  129  0.1327   (129.0/972.0)
1      16   34   0.32     (16.0/50.0)
Total  859  163  0.1419   (145.0/1022.0)]

### Step 6: Save the model for deployment

In [30]:
model_dir = "models_h2o"
native_path = h2o.save_model(leader, path=model_dir, force=True)
mojo_path   = leader.download_mojo(path=model_dir, get_genmodel_jar=True)
print(native_path, mojo_path)

/Users/christinadong/Documents/MLops/auto_ml/models_h2o/GBM_grid_1_AutoML_6_20250816_194941_model_188 /Users/christinadong/Documents/MLops/mlops-final/models/models_h2o/GBM_grid_1_AutoML_6_20250816_194941_model_188.zip


### Step 7: Get predictions on test set

In [31]:
pred = leader.predict(test_h2o)  # columns: p0, p1, predict
pred.head()
# Optionally join predictions back to original rows:
pred_with_labels = test_h2o.cbind(pred)
pred_with_labels.head()

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke,predict,p0,p1
Male,63,0,0,Yes,Private,Rural,78.23,34.8,never smoked,0,0,0.970624,0.0293761
Female,43,0,0,Yes,Private,Urban,86.67,33.3,never smoked,0,0,0.994789,0.00521058
Female,23,0,0,No,Private,Urban,126.67,28.7,smokes,0,0,0.998116,0.00188383
Female,21,0,0,No,Private,Urban,208.17,24.9,never smoked,0,0,0.997395,0.00260484
Male,67,0,0,Yes,Private,Rural,113.34,26.3,formerly smoked,0,1,0.948039,0.0519612
Male,0.16,0,0,No,children,Urban,114.71,17.4,Unknown,0,0,0.997885,0.0021152
Female,47,0,0,Yes,Private,Urban,210.95,50.1,Unknown,0,0,0.973011,0.0269893
Female,20,0,0,No,Govt_job,Rural,73,20.8,never smoked,0,0,0.99832,0.00167975
Female,15,0,0,No,Private,Rural,79.2,22.4,never smoked,0,0,0.998017,0.00198261
Female,2,0,0,No,children,Rural,100.66,18.5,Unknown,0,0,0.998375,0.00162478


### Step 8: Change at least two features and re-validate

In [38]:
changed = test_h2o[:] 
num_cols = [c for c in x if test_pd[c].dtype != "object"]

# pick two numeric features to perturb
feat_a, feat_b = ("bmi", "avg_glucose_level") if {"bmi","avg_glucose_level"}.issubset(set(num_cols)) else (num_cols[0], num_cols[1])

# add small noise / shift  (tune scale as you like)
changed[feat_a] = changed[feat_a] * 1.10           # +10%
changed[feat_b] = changed[feat_b] + 5              # +5 absolute units

perf_changed = leader.model_performance(changed)

# Find the row with maximum F1
test_f1_opt = perf_changed.F1()
best_f1_row = max(test_f1_opt, key=lambda x: x[1])  # x[1] = F1 value
test_f1_opt = best_f1_row[1]
print("Changed data metrics:", {
    "AUC": perf_changed.auc(),
    "LogLoss": perf_changed.logloss(),
    "Best F1": test_f1_opt
})

Changed data metrics: {'AUC': 0.8422325102880658, 'LogLoss': 0.16061327593965388, 'Best F1': 0.3401360544217687}


### Step 9: Quick monitoring hooks you can show

In [41]:
import pandas as pd, datetime as dt

def summarize_perf(perf_obj, tag):
    test_f1_opt = perf_obj.F1()
    best_f1_row = max(test_f1_opt, key=lambda x: x[1]) 
    test_f1_opt = best_f1_row[1]
    return {
        "when": dt.datetime.utcnow().isoformat(),
        "tag": tag,
        "auc": perf_obj.auc(),
        "logloss": perf_obj.logloss(),
        "f1_best": test_f1_opt
    }

logs = []
logs.append(summarize_perf(perf, "baseline_test"))
logs.append(summarize_perf(perf_changed, "changed_numeric"))  # if you ran Option A
pd.DataFrame(logs).to_csv("automl_scoring_log.csv", index=False)


In [42]:
log = pd.read_csv("automl_scoring_log.csv")
log

,when,tag,auc,logloss,f1_best
0,2025-08-17T01:05:35.821397,baseline_test,0.837356,0.161674,0.319249
1,2025-08-17T01:05:35.821468,changed_numeric,0.842233,0.160613,0.340136
